# Standard probability density functions

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon, norm
import ipywidgets as widgets

In [35]:
class PDFwidget:
    def __init__(self, distribution, value_range):
        self.distribution = distribution
        self.value_range = value_range
        self.out = widgets.Output(layout=dict(height='550px'))
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot()
        plt.close()
        self._build()
        self.param_sliders = []

    def _build(self):
        self.button = widgets.Button(description='Draw random sample', layout=dict(width='25%'))
        self.size = widgets.Dropdown(
            options=[('1', 1), 
                     ('10', 10),
                     ('100', 100),
                     ('1000', 1000),
                     ('10000', 10000)],
            value=1,
            description='Sample size:'
        )
        self.plot_pdf = widgets.Checkbox(
            value=True,
            description='Show pmf',
            disabled=False,
            indent=False
        )
        self.plot_sample = widgets.Checkbox(
            value=False,
            description='Show random sample',
            disabled=False,
            indent=False
        )
        self.button.on_click(self.on_button_clicked)
        self.plot_pdf.observe(self.on_button_clicked, names='value')
        self.plot_sample.observe(self.on_button_clicked, names='value')

    def add_param_slider(self, name, slider, func):
        self.param_sliders.append((name, slider, func))
        slider.observe(self.on_button_clicked, names='value')

    def on_button_clicked(self, b):
        with self.out:
            self.out.clear_output(wait=True)
            self.ax.clear()
            dist = self.distribution(**{name: func(slider.value) for (name, slider, func) in self.param_sliders})
            values = np.linspace(self.value_range[0], self.value_range[1], 1000)
            if self.size.value > 1 and self.plot_sample.value:
                sample = dist.rvs(size=self.size.value).reshape(-1, 1)
                self.ax.hist(sample, density=True,
                             bins=np.arange(self.value_range[0],
                                              self.value_range[1], 0.5)
                            )
            elif self.plot_sample.value:
                self.ax.scatter(dist.rvs() + 0.05, 0, marker='o')
            if self.plot_pdf.value:
                xs = values
                ys = dist.pdf(xs)
                self.ax.plot(xs, ys, color='orange', linewidth=3)
            self.ax.set_xlim(self.value_range[0], self.value_range[1])
            self.ax.set_ylim(0, 1.5)
            self.ax.set_xticks(np.arange(self.value_range[0], self.value_range[1]))
            with self.out:
                display(self.fig)
            plt.close()

    def __call__(self):
        self.button.click()
        display(widgets.VBox([self.button, self.size, self.plot_pdf, self.plot_sample]
                             + [slider for (_, slider, __) in self.param_sliders]
                             + [self.out,]))

## Exponential distribution

The exponential distribution is commonly met in applications
to model life span. For instance, it can be used to model
the life span of a light bulb. It takes a single parameter
$\lambda>0$, is denoted $\text{Exp}(\lambda)$, and
its probability density function
takes the form of,
\begin{equation}
    f_X(x; \lambda) = \lambda\exp(-\lambda x), \quad \lambda\in\mathbb{R}^+.
\end{equation}

The exponential distribution is also often used in what is
called queining theory: the modelling of people who stand in a queue.
One can often model the time between two arrivals in a queue as
an exponential distribution.

In [36]:
lamb_sel = widgets.FloatSlider(
    value=0.25,
    min=0,
    max=2,
    step=.05,
    description=r'$\lambda$',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

widget = PDFwidget(expon, (0, 10))
widget.add_param_slider('scale', lamb_sel, lambda x: 1 / x)
widget()

## Normal distribution

The normal distribution is of fundamental importance. Indeed,
it occurs as a limiting distribution (we will define what is 
meant by a limit of a distribution in Week 5) of sample averages
under rather mild assumptions. This is known as the Central
Limit Theorem.

The normal distribution takes two parameters, $\mu\in\mathbb{R}$,
called the mean of the distribution, and $\sigma\in\mathbb{R}^+$,
the standard deviation. It is denoted $\mathcal{N}(\mu, \sigma^2)$
and its probability density function takes the form of,
\begin{equation}
    f_X(x; \mu, \sigma) = 
    \frac{1}{\sqrt{2\pi}\sigma}
    \exp\left\{-\frac{(x-\mu)^2}{2\sigma^2}\right\}.
\end{equation}

In [37]:
mu_sel = widgets.FloatSlider(
    value=0.25,
    min=-5,
    max=5,
    step=.05,
    description=r'$\mu$',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

sigma_sel = widgets.FloatSlider(
    value=0.3,
    min=0.25,
    max=2,
    step=.05,
    description=r'$\sigma$',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

widget = PDFwidget(norm, (-10, 10))
widget.add_param_slider('loc', mu_sel, lambda x: x)
widget.add_param_slider('scale', sigma_sel, lambda x: x)
widget()